<a href="https://colab.research.google.com/github/AK-Github-0/TTS-Finetuning-for-Arabic/blob/main/Finetuning_XTTS_on_SCC22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Clone XTTS repo
!git clone https://github.com/coqui-ai/XTTS-v2.git


# Install required packages manually
!pip install TTS==0.22.0
!pip install git+https://github.com/openai/whisper.git
!pip install git+https://github.com/resemble-ai/monotonic_align.git
!pip install numpy scipy torch torchaudio pandas librosa tensorboard matplotlib unidecode
!pip install datasets


Cloning into 'XTTS-v2'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'XTTS-v2'
/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 78.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 257.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 83.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 13.3 MB/s eta 0:00:00
  Preparing metadata (set

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-dfeazr6t
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-dfeazr6t
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803706 sha256=0a64cdf4bbb2ebc0edd93eb768d13a3938e8da9637e470ef32cc7d1cc0a67b7d
  Stored in directory: /tmp/pip-ephem-wheel-cache-qplso8lb/wheels/1f/1d/98/9583695e6695a6ac0ad42d87511097dce5ba486647dbfecb0e
Successfully built openai-whisper
  Cloning https://github.com/resemble-ai/monotonic_align.git to /tmp/pip-req-build-eqigf0gk
  Running command git clone --filter=blob:none --quiet https://github.com/resemble-ai/monotonic_align.git /tmp/pip-req-build-eqigf0gk
  Resolve

In [7]:
# Optional: for fine-tuning, you typically start from a base model
!wget https://coqui.gateway.scarf.sh/XTTS-v2/xtts_v2.pth -O ./pretrained/xtts_v2.pth


./pretrained/xtts_v2.pth: No such file or directory


In [39]:
!pip install git+https://github.com/coqui-ai/TTS

  Cloning https://github.com/coqui-ai/TTS to /tmp/pip-req-build-mvmhwm6n
  Running command git clone --filter=blob:none --quiet https://github.com/coqui-ai/TTS /tmp/pip-req-build-mvmhwm6n
  Resolved https://github.com/coqui-ai/TTS to commit dbf1a08a0d4e47fdad6172e433eeb34bc6b13b4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 3.6 MB/s eta 0:00:00


In [15]:
!pip install -q datasets huggingface_hub soundfile

from huggingface_hub import hf_hub_download
import os, tarfile, soundfile as sf


In [21]:
import os
from huggingface_hub import hf_hub_download

os.makedirs("scc22_data", exist_ok=True)

# List of parquet files inside the 'data' folder in the repo
files = [
    "test-00000-of-00006.parquet",
    "test-00001-of-00006.parquet",
    "test-00002-of-00006.parquet",
    "test-00003-of-00006.parquet",
    "test-00004-of-00006.parquet",
    "test-00005-of-00006.parquet",
]

for filename in files:
    print(f"Downloading {filename}...")
    path = hf_hub_download(
        repo_id="MohamedRashad/SCC22",
        filename=f"data/{filename}",
        repo_type="dataset",
        local_dir="scc22_data",
        local_dir_use_symlinks=False
    )
    print(f"Downloaded to: {path}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


test-00000-of-00006.parquet:   0%|          | 0.00/414M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Downloaded to: scc22_data/data/test-00000-of-00006.parquet


test-00001-of-00006.parquet:   0%|          | 0.00/353M [00:00<?, ?B/s]

...

test-00004-of-00006.parquet:   0%|          | 0.00/478M [00:00<?, ?B/s]

Downloaded to: scc22_data/data/test-00004-of-00006.parquet


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


test-00005-of-00006.parquet:   0%|          | 0.00/540M [00:00<?, ?B/s]

Downloaded to: scc22_data/data/test-00005-of-00006.parquet


In [30]:
import pandas as pd
import glob
import os # Import os to check directory contents if needed

# Read all parquet files into one DataFrame
all_files = glob.glob("scc22_data/data/*.parquet")

# Add a check to see if files were found
if not all_files:
    print("No parquet files found in 'scc22_data'. Please check if the download was successful.")
    # You might want to add code here to re-attempt download or exit
else:
    print(f"Found {len(all_files)} parquet files.")
    df_list = [pd.read_parquet(f) for f in all_files]
    # Use pd.concat() to concatenate the list of DataFrames
    full_df = pd.concat(df_list)

    print(full_df.head())
    print(f"Total rows: {len(full_df)}")

Found 6 parquet files.
                                               audio   
0  {'bytes': b'RIFF\xfc\xa2\t\x00WAVEfmt \x10\x00...  \
1  {'bytes': b'RIFFT\xb0\x10\x00WAVEfmt \x10\x00\...   
2  {'bytes': b'RIFF\xcc,\n\x00WAVEfmt \x10\x00\x0...   
3  {'bytes': b'RIFF\xf4&\t\x00WAVEfmt \x10\x00\x0...   
4  {'bytes': b'RIFFlk\x10\x00WAVEfmt \x10\x00\x00...   

                                                text   
0       لكن متأخرين كمان إضافة لذلك لأنها شركة جديدة  \
1  There still يعني جالسين يتعاملون مثلا Lucid كل...   
2    يعني من ذا الOne shield من قدام ومن وراء والسقف   
3           وقزاز مختلف يعني لأنه مضلل ومن الكلام ذا   
4     So  الناس اللي يقدروا يسووا هذي الخدمات قليلين   

                                        cleaned_text environment_type   
0       لكن متاخرين كمان اضافة لذلك لانها شركة جديدة            Clean  \
1  there still يعني جالسين يتعاملون مثلا lucid كل...            Clean   
2   يعني من ذا ال one shield من قدام ومن وراء والسقف            Noisy   
3          

In [31]:
from datasets import Dataset

dataset = Dataset.from_pandas(full_df)
print(dataset)


Dataset({
    features: ['audio', 'text', 'cleaned_text', 'environment_type', 'speaker_id', 'speaker_gender', '__index_level_0__'],
    num_rows: 3296
})


In [40]:
import json

config = {
    "output_path": "/content/XTTS-v2/outputs/scc22_finetune",
    "datasets": [
        {
            "name": "scc22",
            "path": "/content/XTTS-v2/data/scc22"
        }
    ],
    "num_loader_workers": 2,
    "run_name": "scc22_xtts",
    "use_pretrained": True,
    "pretrained_path": "/content/XTTS-v2/pretrained/xtts_v2.pth",
    "batch_size": 4,
    "eval_batch_size": 4,
    "epochs": 50,
    "save_every_n_steps": 500,
    "eval_every_n_steps": 250,
    "save_n_best_checkpoints": 3,
    "mixed_precision": True
}

with open("scc22_config.json", "w") as f:
    json.dump(config, f, indent=4)


In [41]:
!python3 scripts/train_xtts.py --config_path scc22_config.json


python3: can't open file '/content/scripts/train_xtts.py': [Errno 2] No such file or directory


In [42]:
from TTS.api import TTS

model_dir = "/content/XTTS-v2/outputs/scc22_finetune"
tts = TTS(model_path=model_dir)


# Inference
output_path = "/content/output.wav"
tts.tts_to_file(
    text="مرحباً بك في عالم تحويل النص إلى كلام باستخدام إكس تي تي إس",
    file_path=output_path,
    speaker_wav="/content/XTTS-v2/data/scc22/wavs/0.wav"  # Use any sample
)


TypeError: expected str, bytes or os.PathLike object, not NoneType